In [ ]:
#importing the required packages
import nltk
nltk.download('all')
import re
import string
import gensim 
from gensim.parsing.preprocessing import remove_stopwords
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from nltk import pos_tag,word_tokenize,ne_chunk
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame

from nltk.corpus import wordnet,stopwords
import spacy
import nltk
nltk.download('punkt')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
from google.colab import files
uploaded=files.upload()
#sample = open("COVID19_wikipedia article.txt", "r") 
sample = open("c.txt", "r")
s = sample.read() 
s=s.replace("COVID 19","coronavirus")
#s='''The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.'''


Saving c.txt to c.txt


In [ ]:
def stem_sentence(sentence):
  words=word_tokenize(sentence)
  #lemmatizer = WordNetLemmatizer()
  
  
  stemmer = SnowballStemmer("english")
  new_words=[]
  for i in words:
    new_words.append(stemmer.stem(i))
    new_words.append(" ")
  return "".join(new_words)  
  


def clean_sentence(sentence, stopwords=True):
    
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    
    
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
   
    
    return sentence
def get_cleaned_sentences(sents,stopwords=True):    
    
    cleaned_sentences=[]

    for i in sents:
        
        cleaning=clean_sentence(i,stopwords)
        cleaned=stem_sentence(cleaning)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences


In [ ]:
def create_document_term_matrix(sen,vectorizer):
  doc_term_matrix=vectorizer.fit_transform(sen)
  return DataFrame(doc_term_matrix.toarray(), columns=vectorizer.get_feature_names())
  

In [ ]:
def calculate_cosine_similarity(df_list,sentences,question):
  a=[]
  for i in range(len(df_list)-1):
    sim=1 - spatial.distance.cosine(df_list[i], question)
    t=(sim,sentences[i])
    a.append(t)
  a.sort(reverse=True)
  n=[]
  for i in range(3):
    n.append(a[i][1])
    print("*",n[i])
   
    
  return n  


In [ ]:
def determineQuestionType( question):
        questionTaggers = ['WP','WDT','WP$','WRB']
        question_POS = pos_tag(word_tokenize(question.lower()))
        
        question_Tags=[]
        for token in question_POS:
            if token[1] in questionTaggers:
              question_Tags.append(token)
                
                
        if len(question_Tags)==1 and question_Tags[0][0]!= 'what' :
          return True
        else:
          return False  

In [ ]:
from nltk.util import ngrams
def n_gram_similarity(question,n):
  q=list(ngrams(word_tokenize(question.lower()),1))
  a=0
  b=0
  c=0
  t=[]
  for i in q:
    if i in list(ngrams(word_tokenize(n[0].lower()),1)):
      a=a+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[1].lower()),1)):
      b=b+1
  for i in q:
    if i in list(ngrams(word_tokenize(n[2].lower()),1)):
      c=c+1        
  d=max(a,b,c)
  if a == d:
    t.append(n[0])
  if b == d:
    t.append(n[1]) 
  if c ==d:
    t.append(n[2])
  print()  
  #print("Selected Sentence:",t[0])  
  return t  
  

In [ ]:
def answertype(question):
  nlp = spacy.load('en_core_web_sm')

  if (determineQuestionType(question)):
    t='DESCRIPTIVE'
    flag=0
    word=word_tokenize(question.lower())
  
    if 'who' in word:
      t='PERSON'
    elif 'where' in word:
      t='GPE'
    elif 'how' in word and 'many' in word and  'age' in word or 'duration' in word or 'long' in word or 'days'in word or 'years' in word or'months' in word:
      t='DATE' 
    elif 'how' in word and 'many' in word :
       t = 'CARDINAL'  
    elif 'when' in word  or 'age' in word or 'period' in word or 'duration' in word  or 'old' in word or 'long' in word:
      t='DATE'
    elif 'how' in word  and 'long' in word or 'often' or 'age' in word or 'years' in word:
      t='DATE' 
    elif 'what' in word and 'time' in word or 'duration' in word or 'period' in  'word'  :
      t='DATE' 
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    n=n_gram_similarity(question,n)
    #print("Most relevant sentence", n[0])
    #print("ANSWER TYPE:",t)
    key = n[0]
    spdoc = nlp(key)
    entity_type=[]
    for ent in spdoc.ents:
       if ent.label_ == t:
          entity_type.append(ent.text)
    if len(entity_type) == 1:
      print("ANSWER TYPE:", t)
      print("ANSWER:", entity_type[0])  
    if len(entity_type) == 0:
      print("ANSWER TYPE:", t) 
      print(n[0])
    if len(entity_type) > 1:
      print("Answer Type:",t)  
      key_question = question
      q=[]
      spdoc = nlp(key_question)
      for ent in spdoc:
        if ent.pos_ == 'NOUN' or ent.pos_ =='ADJ' :
          q.append(ent.text)
  
      key_answer = n[0]
      a = []
      spd = nlp(key)
      for ent in spd:
        if ent.pos_ == 'NOUN'or ent.pos_ =='ADJ' :
          a.append(ent.text)
  #s=[sentence.index(i) for i in t]
      s=[]
      w=[]
      for i in entity_type:
       s.append(n[0].index(i))
      for i in range(len(s)):
        w.append(0)

    
      for i in q:
        try:
           factor= n[0].index(i)
           for j in range(len(s)):
              w[j]=w[j]+(abs(s[j]-factor))
        except:
           continue    
      m=min(w)
      u=[]
      for i in range(len(s)):
        if w[i] == m:
           #print(entity_type[i])
           u.append(entity_type[i])
      print("ANSWER:",u[0])     
      

  

    
  else:
    t='DESCRIPTIVE'
    print("ANSWER TYPE:",t)
    i=len(df_list)-1  
    n=calculate_cosine_similarity(df_list, sentences,df_list[i])
    #n = n_gram_similarity(question, n)
    for j in n:
         print(j) 

In [ ]:
#question=["What preventive measures should I take?","How does the virus spread?","How many days is the incubation period of the virus?","When was COVID-19 declared a health emergency?","When was COVID-19 declared a pandemic?","How many deaths have occured in various countries?","How apart should people stay?","What is the time between exposure to onset of symptoms?","Where was COVID-19 first identified?","Is it necessary to cover my face?","Should I cover my face?","When the did the first case of COVID-19 occur in Delhi ?","what was the age of the first patient in Delhi?", "Where has the  virus been found in wastewater?","How many cases have been reported?","What facemasks should I use?","What method is used for diagnosis?","How old was the first patient in Delhi?","What about the vaccine?","How many days is the incubation period of the virus?","How long do the affected persons remain infectious?","Does loss of smell also occur?","When did World health organization declare coronavirus a world pandemic?","How many people have recovered?"]
#question=["What is Zealandia?", "How many years ago did Zealandia subside?","In which ocean is Zealandia submerged?","When were permits for oil exploration in the Great South Basin issued?"]
#question=["Where did Black Death originate?","How did the black death make it to the Mediterranean and Europe?","How much of the European population did the black death kill?","When did the world's population finally recover from the black death?","For how long did the plague stick around?","When did the Black Death originate in Central Asia?","In what year did the Black Death reach the Mediterranean?"]
#question=["How many people were in French North American Colonies?","Who fought in the French and Indian war?","How many people were in French North American Colonies?","How many people were in British North American Colonies?","When was the French and Indian War?"]
question=["Where was the virus first identified?","When was the virus first identified?","When did the first case occur in Delhi?","How many days is the incubation period of the virus?","When was COVID-19 declared a health emergency?","When was COVID-19 declared a pandemic?","How apart should people stay?","How many people have recovered till now?","How many cases have been reported?","How many deaths have occured in various countries?","In how many World Health Organization zones local transmission has occuerd?","Cases have been reported in how many countries?","How many countries have reported cases?","How long should I wash my hands?","What are the common symptoms of COVID-19?","What type of masks should I wear?"]

#question=["Which name is also used to describe the Amazon rainforest in English?","How many square kilometers of rainforest is covered in the basin?","How many nations control this region in total?","How many nations contain Amazonas in their names?","What percentage does the Amazon represents in rainforests on the planet?","How many square kilometers is the Amazon Basin?","How many nations are within the Amazon Basin?","How many tree species are there in the amazon tropical rain forest?"]
for j in question:
  que=sent_tokenize(j)
  sentences=sent_tokenize(s)
  #q contains a list of cleaned sentence tokens of question
  q=get_cleaned_sentences(que,stopwords=True)
  #preprocessed contains a list of cleaned sentence tokens of the reference text
  preprocessed=get_cleaned_sentences(sentences,stopwords=True)
  
  preprocessed.append(q[0])
  i=len(preprocessed)-1
  print("QUESTION:",j)
  
  tfidf_vect=TfidfVectorizer()
  df=create_document_term_matrix(preprocessed,tfidf_vect) 
  df_list = df.values.tolist()
  answertype(j)
  print()




QUESTION: Where was the virus first identified?
* It was first identified in December 2019 in Wuhan, China, and has resulted in an ongoing pandemic.
* On surfaces the amount of active virus decreases over time until it can no longer cause infection, and surfaces are thought not to be the main way the virus spreads.
* People can transmit the virus without showing symptoms, but it is unclear how often this happens.

Answer Type: GPE
ANSWER: Wuhan

QUESTION: When was the virus first identified?
* It was first identified in December 2019 in Wuhan, China, and has resulted in an ongoing pandemic.
* On surfaces the amount of active virus decreases over time until it can no longer cause infection, and surfaces are thought not to be the main way the virus spreads.
* People can transmit the virus without showing symptoms, but it is unclear how often this happens.

ANSWER TYPE: DATE
ANSWER: December 2019

QUESTION: When did the first case occur in Delhi?
* Transmission without symptoms does occur